In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())

In [2]:
myTable = soup.find('table',{'class':'wikitable sortable'})
print (myTable.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

In [3]:
titles = []
for title in myTable.findAll('th'):
    titles.append(title.text)
print(titles)

['Postcode', 'Borough', 'Neighbourhood\n']


In [4]:
rowlen = len(myTable.findAll('tr'))
np.arange(0,rowlen+1,1)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [7]:
rowlen = len(myTable.findAll('tr'))
df = pd.DataFrame(index=np.arange(0,rowlen+1,1),columns=list(titles))
row_marker = 0
for row in myTable.findAll('tr'): #rows in tr
    column_marker = 0
    columns = row.findAll('td') # columns in td
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text() #iat returns value from row and col position
        column_marker +=1
    row_marker +=1

df.head()

,Postcode,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


There are no Boroughs with Neighbourhoods containing "Not assigned value".

In [14]:
df.loc[(df["Borough"] != 'Not assigned') & (df["Neighbourhood\n"] == 'Not assigned')]

,Postcode,Borough,Neighbourhood


In [15]:
df.dropna(axis = 0,inplace=True)
# drop rows with NaN values

drop postcodes wich have to Borough and Neighbourhood assigned

In [16]:
df = df[df.Borough != 'Not assigned']

In [17]:
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n
6,M5A,Downtown Toronto,Regent Park\n
7,M6A,North York,Lawrence Heights\n


Remove "\n" from end of string.

In [18]:
df = df.replace('\n','',regex=True)

In [19]:
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [20]:
df.describe()

,Postcode,Borough,Neighbourhood
count,212,212,212
unique,103,11,210
top,M9V,Etobicoke,St. James Town
freq,8,45,2


In [22]:
df.loc[df["Postcode"] == 'M5A']

,Postcode,Borough,Neighbourhood
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


In [23]:
print(df.columns)

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')


The neighbourhood column name needs to be changed as it contains "\n"

In [24]:
df.rename(columns={'Neighbourhood\n':'Neighbourhood'},inplace = True)

Dataframe is grouped by Postcode and Borough by joining the Neighbourhood strings by comma

In [26]:
df2 = df.groupby(["Postcode","Borough"])["Neighbourhood"].apply(', '.join).reset_index()

In [28]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
df.shape

(212, 3)